In [1]:
import pandas as pd

# 한글 폰트 깨짐 방지
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

#에러 무시
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from sklearn.tree import export_graphviz
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import scale, minmax_scale, robust_scale

import xgboost as xgb

import seaborn as sns

plt.rcParams['font.family'] = 'NanumGothic'


In [255]:
# basic = pd.read_csv('타자 기본.csv', encoding = 'CP949')
hts = pd.read_csv('hts.csv', encoding = 'CP949')

In [256]:
hts['타격결과'].unique()

array(['플라이', '1루타', '번트안타', '땅볼아웃', '홈런', '2루타', '파울플라이', '3루타', '직선타',
       '희생번트', '내야안타(1루타)', '희생플라이', '병살타', '인필드플라이', '번트아웃', '야수선택',
       '삼중살타'], dtype=object)

## 안타 여부 0, 1로 나누기 / 장타여부 1~4

- 먼저 hts를 train/test로 나누고 : 10만개 / 2만개
- 안타 여부 0, 1로 분류
- train 후 예측

In [257]:
hts['안타여부'] = 0
hts['장타여부'] = 0
# hts['안타여부'] = hts.loc[(hts['타격결과'] == '1루타') == 1]

In [258]:
# 안타가 된 타구만 1로 
hts['안타여부'][(hts['타격결과'] == '1루타')] = 1
hts['안타여부'][(hts['타격결과'] == '2루타')] = 1
hts['안타여부'][(hts['타격결과'] == '3루타')] = 1
hts['안타여부'][(hts['타격결과'] == '홈런')] = 1
hts['안타여부'][(hts['타격결과'] == '내야안타(1루타)')] = 1

# 장타가 된 타구
hts['장타여부'][(hts['타격결과'] == '1루타')] = 1
hts['안타여부'][(hts['타격결과'] == '내야안타(1루타)')] = 1
hts['장타여부'][(hts['타격결과'] == '2루타')] = 2
hts['장타여부'][(hts['타격결과'] == '3루타')] = 3
hts['장타여부'][(hts['타격결과'] == '홈런')] = 4

# 번트 드랍
# hts.drop(index = hts.loc[(hts['타격결과'] == '번트아웃')|
#                   (hts['타격결과'] == '번트안타')|
#                   (hts['타격결과'] == '희생번트')], axis = 0)

In [259]:
hts.loc[(hts['타격결과'] == '희생번트')|
       (hts['타격결과'] == '번트아웃')|
       (hts['타격결과'] == '번트안타')]

,연도,경기코드,타구코드,선수코드,타자 팀 코드,이닝,타구속도,발사각도,타격결과,투구구속,해당구장,안타여부,장타여부
4,2018,20180324HHWO0,180324_142050,68730,HH,2,54.11,16.4,번트안타,134.07,고척,0,0
48,2018,20180324KTHT0,180324_141148,70410,HT,1,35.46,10.5,희생번트,140.99,광주,0,0
80,2018,20180324KTHT0,180324_162220,76849,HT,6,40.65,-36.4,희생번트,128.62,광주,0,0
194,2018,20180324SSOB0,180324_150949,62415,SS,4,43.23,-7.0,희생번트,145.92,잠실,0,0
442,2018,20180325SSOB0,180325_155937,72466,SS,7,33.17,-68.9,희생번트,139.04,잠실,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46213,2019,20190522HHSS0,190522_230544,79402,SS,12,47.50,-6.7,희생번트,131.23,대구,0,0
46711,2019,20190524LGLT0,190524_183221,79150,LG,1,39.09,11.8,희생번트,136.53,사직,0,0
51304,2019,20190616LGOB0,190616_170516,79150,LG,1,47.99,-3.6,희생번트,137.24,잠실,0,0
56540,2019,20190714SSLG0,190714_201054,61186,LG,8,56.62,-39.9,희생번트,136.58,잠실,0,0


## 번트 drop
1. 번트 행 삭제
2. 삭제 후 3만개 test로 추출 / 9만개 train으로 추출

  - 조건 맞는 행 삭제하는 방법 알려주실분

In [260]:
# 번트 행 삭제

hts = hts.loc[(hts['타격결과'] != '번트안타')&
       (hts['타격결과'] != '번트아웃')&
       (hts['타격결과'] != '희생번트')]

hts.reset_index(inplace = True, drop = True)

hts = hts[['발사각도', '타구속도', '타격결과', '안타여부', '장타여부']]
hts

,발사각도,타구속도,타격결과,안타여부,장타여부
0,42.7,131.50,플라이,0,0
1,9.9,135.18,1루타,1,1
2,2.1,152.41,1루타,1,1
3,13.5,113.72,1루타,1,1
4,3.5,161.90,1루타,1,1
...,...,...,...,...,...
120409,-4.9,141.23,땅볼아웃,0,0
120410,31.8,101.09,1루타,1,1
120411,33.4,146.84,홈런,1,4
120412,23.9,146.12,홈런,1,4


In [261]:
# 임의로 3만개 / 9만414개 나누기

# test
test = []
for i in range(0, len(hts), 4):
    test.append(i)

hts_test = hts.iloc[test].reset_index(drop = True)

In [262]:
hts_test

,발사각도,타구속도,타격결과,안타여부,장타여부
0,42.7,131.50,플라이,0,0
1,3.5,161.90,1루타,1,1
2,61.1,127.35,플라이,0,0
3,26.4,128.37,플라이,0,0
4,17.6,116.31,1루타,1,1
...,...,...,...,...,...
30099,22.6,125.73,2루타,1,2
30100,8.8,117.65,땅볼아웃,0,0
30101,63.3,120.02,플라이,0,0
30102,69.4,96.34,플라이,0,0


In [263]:
# train
train = []
for i in range(0, len(hts), 4):
    hts.drop(index = i, inplace = True)

In [264]:
hts.reset_index(drop = True)
hts_train = hts
hts_train.reset_index(inplace = True, drop = True)

In [265]:
hts_train.to_csv('hts_train.csv', index = False, encoding = 'CP949')
hts_test.to_csv('hts_test.csv', index = False, encoding = 'CP949')

In [266]:
hts_train

,발사각도,타구속도,타격결과,안타여부,장타여부
0,9.9,135.18,1루타,1,1
1,2.1,152.41,1루타,1,1
2,13.5,113.72,1루타,1,1
3,0.9,105.03,땅볼아웃,0,0
4,26.6,115.52,플라이,0,0
...,...,...,...,...,...
90305,-24.7,86.89,땅볼아웃,0,0
90306,-4.9,141.23,땅볼아웃,0,0
90307,31.8,101.09,1루타,1,1
90308,33.4,146.84,홈런,1,4


## 그냥 EDA

In [100]:
# 플라이
hts_fly = hts.loc[(hts['타격결과'] == '플라이')|
                  (hts['타격결과'] == '희생플라이')] 

hts_ffly = hts.loc[(hts['타격결과'] == '파울플라이')] 

hts_infly = hts.loc[(hts['타격결과'] == '인필드플라이')]


# 땅볼
hts_ground = hts.loc[(hts['타격결과'] == '땅볼아웃')|
                     (hts['타격결과'] == '병살타')|
                     (hts['타격결과'] == '삼중살타')|
                     (hts['타격결과'] == '야수선택')] 

#장타
hts_hr = hts.loc[(hts['타격결과'] == '홈런')] 
hts_2 = hts.loc[(hts['타격결과'] == '2루타')] 
hts_3 = hts.loc[(hts['타격결과'] == '3루타')] 

# 안타성
hts_line = hts.loc[(hts['타격결과'] == '직선타')]

# 1루타
hts_1 = hts.loc[(hts['타격결과'] == '1루타')]

hts_1in = hts.loc[(hts['타격결과'] == '내야안타(1루타)')]


# 번트
hts_bunt = hts.loc[(hts['타격결과'] == '번트아웃')|
                  (hts['타격결과'] == '번트안타')|
                  (hts['타격결과'] == '희생번트')]

In [102]:
hts_result_vel = pd.DataFrame(hts.groupby('타격결과')['타구속도'].agg('mean').sort_values()).reset_index()
hts_result_ang = pd.DataFrame(hts.groupby('타격결과')['발사각도'].agg('mean').sort_values()).reset_index()

In [103]:
hts_result_vel

,타격결과,타구속도
0,희생번트,36.291004
1,번트아웃,47.452500
2,번트안타,49.841667
3,인필드플라이,111.592598
4,파울플라이,113.005253
5,야수선택,117.096923
6,내야안타(1루타),127.464066
7,땅볼아웃,128.049032
8,플라이,131.195242
9,병살타,132.850467


In [104]:
hts_result_ang

,타격결과,발사각도
0,삼중살타,-13.650000
1,희생번트,-13.031381
2,번트안타,-9.900000
3,야수선택,-8.244231
4,번트아웃,-7.763636
5,땅볼아웃,-7.601705
6,병살타,-7.090470
7,내야안타(1루타),-4.039613
8,1루타,8.119682
9,직선타,9.322238


In [105]:
hts[['타구속도', '발사각도', '타격결과', '안타여부', '장타여부']]

,타구속도,발사각도,타격결과,안타여부
0,131.50,42.7,플라이,0
1,135.18,9.9,1루타,1
2,152.41,2.1,1루타,1
3,113.72,13.5,1루타,1
4,54.11,16.4,번트안타,0
...,...,...,...,...
120740,141.23,-4.9,땅볼아웃,0
120741,101.09,31.8,1루타,1
120742,146.84,33.4,홈런,1
120743,146.12,23.9,홈런,1
